## Import the modules

In [3]:
import fastf1 as ff1 
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score
import gradio as gr
import seaborn as sns
import numpy as np
import gradio as gr

c:\Users\adhen\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Make a Dictionary on the file locations

In [4]:
grand_prix_files = {
    "Austrian Grand Prix": "race_stints_2024/austrian_grand_prix_stints.csv",
    "British Grand Prix": "race_stints_2024/british_grand_prix_stints.csv",
    "Bahrain Grand Prix": "race_stints_2024/bahrain_grand_prix_stints.csv",
    "Saudi Arabian Grand Prix": "race_stints_2024/saudi_arabian_grand_prix_stints.csv",
    "Australian Grand Prix": "race_stints_2024/australian_grand_prix_stints.csv",
    "Emilia Romagna Grand Prix": "race_stints_2024/emilia_romagna_grand_prix_stints.csv",
    "Monaco Grand Prix": "race_stints_2024/monaco_grand_prix_stints.csv",
    "Spanish Grand Prix": "race_stints_2024/spanish_grand_prix_stints.csv",
    "Canadian Grand Prix": "race_stints_2024/canadian_grand_prix_stints.csv",
    "French Grand Prix": "race_stints_2024/french_grand_prix_stints.csv",
    "Hungarian Grand Prix": "race_stints_2024/hungarian_grand_prix_stints.csv",
    "Belgian Grand Prix": "race_stints_2024/belgian_grand_prix_stints.csv",
    "Dutch Grand Prix": "race_stints_2024/dutch_grand_prix_stints.csv",
    "Italian Grand Prix": "race_stints_2024/italian_grand_prix_stints.csv",
    "Singapore Grand Prix": "race_stints_2024/singapore_grand_prix_stints.csv",
    "Japanese Grand Prix": "race_stints_2024/japanese_grand_prix_stints.csv",
    "United States Grand Prix": "race_stints_2024/united_states_grand_prix_stints.csv",
    "Mexican Grand Prix": "race_stints_2024/mexican_grand_prix_stints.csv",
    "Brazilian Grand Prix": "race_stints_2024/brazilian_grand_prix_stints.csv",
    "Qatar Grand Prix": "race_stints_2024/qatar_grand_prix_stints.csv",
    "Abu Dhabi Grand Prix": "race_stints_2024/abu_dhabi_grand_prix_stints.csv",
    "Miami Grand Prix": "race_stints_2024/miami_grand_prix_stints.csv",
    "Las Vegas Grand Prix": "race_stints_2024/las_vegas_grand_prix_stints.csv"
}

## Display the data with Gradio and make it interactive

In [5]:
def optimal_pit_stop_lap(grand_prix):
    print(grand_prix)
    file_path = grand_prix_files[grand_prix]
    data = pd.read_csv(file_path)

    # Mapping the tyre compounds
    compound_map = {
        "SOFT": 1,
        "MEDIUM": 2,
        "HARD": 3,
        "INTERMEDIATE": 4,
        "WET": 5

    }

    data["Compound"] = data["Compound"].map(compound_map)

    # Mapping the drivers
    drivers_map = {
        "ALB": 1,  # Alexander Albon
        "ALO": 2,  # Fernando Alonso
        "BOT": 3,  # Valtteri Bottas
        "HAM": 4,  # Lewis Hamilton
        "HUL": 5,  # Nico Hulkenberg
        "LEC": 6,  # Charles Leclerc
        "PIA": 7,  # Oscar Piastri
        "NOR": 8,  # Lando Norris
        "PER": 9,  # Sergio Perez
        "RIC": 10, # Daniel Ricciardo
        "STR": 11, # Lance Stroll
        "TSU": 12, # Yuki Tsunoda
        "ZHO": 14, # Zhou Guanyu
        "GAS": 15, # Pierre Gasly
        "SAI": 16, # Carlos Sainz
        "VER": 17, # Max Verstappen
        "RUS": 18, # George Russell
        "OCO": 19, # Esteban Ocon
        "SAR": 20, # Logan Sargeant
        "MAG": 21, # Kevin Magnussen
        "LAW": 22, # Liam Lawson
        "COL": 23, # Franco Colapinto
        "DOO": 24, # Jack Doohan
        "BEA": 25  # Oliver Bearman
    }

    data["Driver"] = data["Driver"].map(drivers_map)

    # Drop rows with NaN values in any column
    data = data.dropna()

    x = data[["Stint", "Compound", "Driver"]]
    y = data["Laps"]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor()
    model.fit(x_train, y_train)
    predictions = model.predict(x_test)

    # Calculate residuals
    residuals = np.round(np.abs(y_test - predictions), decimals=2)

    # Combine test data, predictions, and residuals into a DataFrame
    results = x_test.copy()
    results["Actual Laps"] = y_test
    results["Predicted Laps"] = np.round(predictions, decimals=2)
    results["Residuals"] = residuals

    # Find the optimal lap for each driver
    optimal_laps = results.loc[results.groupby("Driver")["Residuals"].idxmin()]

    print("Optimal Laps for Each Driver:")
    print(optimal_laps)

    # Sort the results by Driver and Residuals
    results_sorted = results.sort_values(by=["Residuals"])

    # Pick the top 3 values with the lowest residuals for each driver
    top_3_optimal_laps = results_sorted.groupby("Residuals").head(3).reset_index(drop=True)



    print("Top 3 Optimal Laps for Each Driver:")
    print(top_3_optimal_laps)

    # calculate accuracy
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    print(f'Mean Squared Error: {mse:.2f}')
    print(f'Mean Absolute Error: {mae:.2f}')
    print(f'R2 Score: {r2*100:.2f}%')

    return top_3_optimal_laps.head(3)

iface = gr.Interface(fn=optimal_pit_stop_lap,
                    inputs=gr.Dropdown(
                        choices=list(grand_prix_files.keys()),
                        label="Select Grand Prix"),
                    outputs=gr.DataFrame(),
                    title="F1 Optimal Pit Stop Prediction",
                    description="Predict the optimal lap to pit for the selected Grand Prix",)
iface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7868

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


British Grand Prix
Optimal Laps for Each Driver:
    Stint  Compound  Driver  Actual Laps  Predicted Laps  Residuals
0     1.0         2       1           27           26.75       0.25
5     3.0         2       2           14           13.67       0.33
12    3.0         1       4           14           13.73       0.27
16    1.0         2       6           19           26.51       7.51
33    3.0         4       9            9            9.73       0.73
41    3.0         1      10           14           12.64       1.36
53    3.0         2      11           13           12.63       0.37
61    2.0         2      14            7           10.51       3.51
9     1.0         2      15            1           26.17      25.17
45    2.0         4      16           13           10.54       2.46
30    5.0         2      19           12            5.04       6.96
Top 3 Optimal Laps for Each Driver:
    Stint  Compound  Driver  Actual Laps  Predicted Laps  Residuals
0     1.0         2       1    